In [1]:
from data import load_dataset 
import numpy as np
import os
import pandas as pd
ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, 'data/ml-1m/')

#### User_dict, User_history npy 파일 만들기

In [2]:
convert = pd.read_csv(os.path.join(DATA_DIR, 'ratings_converted.csv'))
convert = convert.applymap(int)
convert = convert.sort_values(by='timestamp', ascending=True)       #이렇게 정리해야 user_dict의 history가 최근순으로 정렬됨? -> 근데 어차피 일관성만 있으면 괜찮을 듯
convert

,userID,itemID,ratings,timestamp
268822,6038,662,4,956703932
88596,6038,489,5,956703954
655460,6038,1715,4,956703954
29800,6038,1403,4,956703977
366450,6038,1453,5,956703977
...,...,...,...,...
415827,5946,2260,4,1046437932
174096,5310,2376,4,1046444711
264155,4956,2506,4,1046454320
371493,4956,1027,5,1046454443


In [3]:
users_dict = {user : [] for user in set(convert["userID"])}

In [4]:
# add (movie, rating) pair to the user dictionary
## add movie rated >= 4 only
convert_gen = convert.iterrows()
users_dict_for_history_len = {user : [] for user in set(convert["userID"])}
for data in convert_gen:
    users_dict[data[1]['userID']].append((data[1]['itemID'], data[1]['ratings']))
    if data[1]['ratings'] >= 4:
        users_dict_for_history_len[data[1]['userID']].append((data[1]['itemID'], data[1]['ratings']))

In [5]:
users_history_lens = [len(users_dict_for_history_len[u]) for u in set(convert["userID"])]

In [6]:
# save each data as npy to save time for future evaluation
np.save("./data/final_user_dict.npy", users_dict)
np.save("./data/final_users_history_len.npy", users_history_lens)

#### load_dataset 함수 확인

In [2]:
train_users_num, train_items_num, train_users_dict, train_users_history_lens, movies_id_to_movies = load_dataset(DATA_DIR, 'train')

Interact Data loading...
Data loading complete!
Data preprocessing...
total users_num : 6039, total items_num : 2819
train_users_num : 4831
Done


In [3]:
eval_users_num, _, test_users_dict, test_users_history_lens, _ = load_dataset(DATA_DIR, 'test')

Interact Data loading...
Data loading complete!
Data preprocessing...
total users_num : 6039, total items_num : 2819
eval_users_num : 1208
Done


In [4]:
# key 값 겹치는지 확인
train_key = train_users_dict.keys()
test_key = test_users_dict.keys()
print(train_key & test_key)
print(len(train_key), len(test_key))

set()
4831 1208


In [5]:
# history_len에 positive rating만 들어간건지 확인
for user_id, history in train_users_dict.items():
    cnt = 0
    for movie_id, rating in history:
        if rating > 3:
            cnt += 1
    
    if cnt != train_users_history_lens[user_id]:
        print(user_id, cnt, train_users_history_lens[user_id])

In [7]:
for user_id, history in test_users_dict.items():
    cnt = 0
    for movie_id, rating in history:
        if rating > 3:
            cnt += 1
    
    if cnt != test_users_history_lens[user_id]:
        print(user_id, cnt, test_users_history_lens[user_id])